In [1]:
import importlib
import models
import helper_funtions
import preprocess
import settings

importlib.reload(settings)
# importlib.reload(models)
importlib.reload(helper_funtions)
importlib.reload(preprocess)

from models.LSTM import LSTM_HumanFi, CNN_LSTM, CNN_BiLSTM_TemporalAttention, CNN_BiLSTM_ChannelAttention, CNN_BiLSTM_DualAttention, CNN_BiLSTM_Attention
from models.RadioNet import RadioNet_NeuralWave
from models.ResNet import ECAResNet1D, ECABasicBlock1D, ResNet1D_JARILWWF, OptResNet1D_JARILWWF, OptECAResNet1D_JARILWWF, CustomResNet1D, CustomECAResNet1D
from models.TemporalConvNet import TemporalConvNet

from helper_funtions import grid_search, run_gridsearch_with_seeds
from preprocess import DataPreprocessor
from settings import folder_path_5ghz_10hz_collected, folder_path_60ghz_collected, folder_path_5ghz_200hz_collected, folder_path_60ghz_external, output_path

In [2]:
input_dim = 52
num_epochs = 500

param_grid = {
    'batch_size': [32],
    'learning_rate': [0.0007], # 0.001,
    'optimizer': ['adam'],
    'mixup_alpha': [0.0, 0.4],
    'smoothing_prob': [0.0, 0.5],
    'model': [],
}

hidden_dims = [64, 128] # 32, 256
num_layers_list = [1, 2]
bidirectional_flags = [False, True]
dropouts = [0.2, 0.5]

for hidden_dim in hidden_dims:
    for num_layers in num_layers_list:
        for bidirectional in bidirectional_flags:
            for dropout in dropouts:
                param_grid['model'].append({
                    'model_class': LSTM_HumanFi,
                    'model_args': {
                        'input_dim': input_dim,
                        'hidden_dim': hidden_dim,
                        'num_layers': num_layers,
                        'bidirectional': bidirectional,
                        'dropout': dropout
                    },
                    'num_epochs': num_epochs
                })

tcn_channel_options = [
    # [32, 64],
    [64, 128],
    [64, 128, 128],
    [64, 128, 256],
    # [128, 256, 512],
]
tcn_kernel_sizes = [2, 3] # 5
tcn_dropout_rates = [0.2, 0.5]

for channels in tcn_channel_options:
    for kernel_size in tcn_kernel_sizes:
        for dropout in tcn_dropout_rates:
            param_grid['model'].append({
                'model_class': TemporalConvNet,
                'model_args': {
                    'num_inputs': input_dim,
                    'num_channels': channels,
                    'kernel_size': kernel_size,
                    'dropout': dropout
                },
                'num_epochs': num_epochs
            })

cnn_lstm_hidden_dims = [64, 128] # 256
cnn_lstm_layers = [1, 2]

for hidden_dim in cnn_lstm_hidden_dims:
    for num_layers in cnn_lstm_layers:
        param_grid['model'].append({
            'model_class': CNN_BiLSTM_Attention,
            'model_args': {
                'input_dim': input_dim,
                'cnn_filters': 64,
                'lstm_units': hidden_dim,
                'num_layers': num_layers,
            },
            'num_epochs': num_epochs
        })

        param_grid['model'].append({
            'model_class': CNN_BiLSTM_TemporalAttention,
            'model_args': {
                'input_dim': input_dim,
                'cnn_channels': 64,
                'lstm_hidden_dim': hidden_dim,
                'lstm_layers': num_layers,
            },
            'num_epochs': num_epochs
        })

        # param_grid['model'].append({
        #     'model_class': CNN_BiLSTM_ChannelAttention,
        #     'model_args': {
        #         'input_dim': input_dim,
        #         'cnn_channels': 64,
        #         'lstm_hidden_dim': hidden_dim,
        #         'lstm_layers': num_layers,
        #     },
        #     'num_epochs': num_epochs
        # })
        #
        # param_grid['model'].append({
        #     'model_class': CNN_BiLSTM_DualAttention,
        #     'model_args': {
        #         'input_dim': input_dim,
        #         'cnn_channels': 64,
        #         'lstm_hidden_dim': hidden_dim,
        #         'lstm_layers': num_layers,
        #     },
        #     'num_epochs': num_epochs
        # })


param_grid['model'] += [
    # {
    #     'model_class': RadioNet_NeuralWave,
    #     'model_args': {'input_dim': input_dim},
    #     'num_epochs': num_epochs,
    #     'data_preprocessor': DataPreprocessor(target_dim=354)
    # },
    
    # the ones commented below are not optimal (proven by early tests)
    # {
    #     'model_class': ECAResNet1D,
    #     'model_args': {'input_channels': input_dim, 'block': ECABasicBlock1D, 'layers': (1, 1, 1, 1)},
    #     'num_epochs': num_epochs
    # },
    # {
    #     'model_class': ECAResNet1D,
    #     'model_args': {'input_channels': input_dim, 'block': ECABasicBlock1D, 'layers': (2, 2, 2, 2)},
    #     'num_epochs': num_epochs
    # },
    # {
    #     'model_class': ResNet1D_JARILWWF,
    #     'model_args': {'input_channels': input_dim, 'layers': [1,1,1,1]},
    #     'num_epochs': num_epochs,
    # },
    # {
    #     'model_class': ResNet1D_JARILWWF,
    #     'model_args': {'input_channels': input_dim, 'layers': [2,2,2,2]},
    #     'num_epochs': num_epochs,
    # },
    
    
    {
        'model_class': OptResNet1D_JARILWWF,
        'model_args': {'input_channels': input_dim, 'layers': [1,1,1,1]},
        'num_epochs': num_epochs,
    },
    # {
    #     'model_class': OptResNet1D_JARILWWF,
    #     'model_args': {'input_channels': input_dim, 'layers': [2,2,2,2]},
    #     'num_epochs': num_epochs,
    # },
    {
        'model_class': OptECAResNet1D_JARILWWF,
        'model_args': {'input_channels': input_dim, 'layers': [1,1,1,1]},
        'num_epochs': num_epochs,
    },
    # {
    #     'model_class': OptECAResNet1D_JARILWWF,
    #     'model_args': {'input_channels': input_dim, 'layers': [2,2,2,2]},
    #     'num_epochs': num_epochs,
    # },
    {
        'model_class': CustomResNet1D,
        'model_args': {'input_channels': input_dim},
        'num_epochs': num_epochs,
    },
    {
        'model_class': CustomECAResNet1D,
        'model_args': {'input_channels': input_dim},
        'num_epochs': num_epochs,
    },
] 

In [3]:
gridsearch_results_df = run_gridsearch_with_seeds(
    gridsearch_func=grid_search,
    gridsearch_args={
        'param_grid': param_grid,
        'folder_path': folder_path_5ghz_200hz_collected,
        'background_subtraction': False,
        'seconds_per_sample': 5,
        'rows_per_second': 200,
    },
    n_seeds=3,
    output_dir=output_path,
    filename="gridsearch_5ghz_coll_200hz"
)


--- Running seed 42 ---
Running 160 configurations...

Processing Model: LSTM_HumanFi with params {'input_dim': 52, 'hidden_dim': 64, 'num_layers': 1, 'bidirectional': False, 'dropout': 0.2}
Started loading data...
Train size: 345, Validation size: 62, Test size: 62
Train labels distribution: Counter({8: 21, 0: 20, 1: 20, 2: 20, 13: 20, 14: 20, 15: 20, 16: 20, 3: 19, 4: 19, 6: 19, 10: 19, 11: 19, 12: 19, 5: 18, 7: 18, 9: 17, 17: 17})
Validation labels distribution: Counter({0: 4, 3: 4, 4: 4, 6: 4, 8: 4, 10: 4, 11: 4, 12: 4, 1: 3, 2: 3, 5: 3, 7: 3, 9: 3, 13: 3, 14: 3, 15: 3, 16: 3, 17: 3})
Test labels distribution: Counter({0: 4, 3: 4, 4: 4, 6: 4, 8: 4, 10: 4, 11: 4, 12: 4, 1: 3, 2: 3, 5: 3, 7: 3, 9: 3, 13: 3, 14: 3, 15: 3, 16: 3, 17: 3})
Done loading data (batch size: 32) in 0:00:00.230075.

Testing Configuration 0: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.0, smoothing_prob=0.0

(Device: cuda) Started training model: LSTM_HumanFi
Early stopping at epoch 216 (no improvem

C:\Users\centu\PycharmProjects\MasterThesis_mmWavePI\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Early stopping at epoch 181 (no improvement in 50 epochs)
Training time:  0:08:21.254241


C:\Users\centu\PycharmProjects\MasterThesis_mmWavePI\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)



Testing Configuration 65: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.0, smoothing_prob=0.5

(Device: cuda) Started training model: TemporalConvNet
Early stopping at epoch 109 (no improvement in 50 epochs)
Training time:  0:06:52.293251


...


Testing Configuration 109: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.0, smoothing_prob=0.5

(Device: cuda) Started training model: TemporalConvNet
Early stopping at epoch 124 (no improvement in 50 epochs)
Training time:  0:04:29.386648


C:\Users\centu\PycharmProjects\MasterThesis_mmWavePI\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)



Testing Configuration 110: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.4, smoothing_prob=0.0

(Device: cuda) Started training model: TemporalConvNet
Early stopping at epoch 188 (no improvement in 50 epochs)
Training time:  0:06:50.634672


C:\Users\centu\PycharmProjects\MasterThesis_mmWavePI\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)



Testing Configuration 111: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.4, smoothing_prob=0.5

(Device: cuda) Started training model: TemporalConvNet
Early stopping at epoch 276 (no improvement in 50 epochs)
Training time:  0:10:01.272417

Processing Model: CNN_BiLSTM_Attention with params {'input_dim': 52, 'cnn_filters': 64, 'lstm_units': 64, 'num_layers': 1}
Started loading data...
Train size: 345, Validation size: 62, Test size: 62
Train labels distribution: Counter({8: 21, 0: 20, 1: 20, 2: 20, 13: 20, 14: 20, 15: 20, 16: 20, 3: 19, 4: 19, 6: 19, 10: 19, 11: 19, 12: 19, 5: 18, 7: 18, 9: 17, 17: 17})
Validation labels distribution: Counter({0: 4, 3: 4, 4: 4, 6: 4, 8: 4, 10: 4, 11: 4, 12: 4, 1: 3, 2: 3, 5: 3, 7: 3, 9: 3, 13: 3, 14: 3, 15: 3, 16: 3, 17: 3})
Test labels distribution: Counter({0: 4, 3: 4, 4: 4, 6: 4, 8: 4, 10: 4, 11: 4, 12: 4, 1: 3, 2: 3, 5: 3, 7: 3, 9: 3, 13: 3, 14: 3, 15: 3, 16: 3, 17: 3})
Done loading data (batch size: 32) in 0:00:00.107527.

Testing Confi

In [3]:
gridsearch_results_df_bgsub = run_gridsearch_with_seeds(
    gridsearch_func=grid_search,
    gridsearch_args={
        'param_grid': param_grid,
        'folder_path': folder_path_5ghz_200hz_collected,
        'background_subtraction': True,
        'seconds_per_sample': 5,
        'rows_per_second': 200,
    },
    n_seeds=3,
    output_dir=output_path,
    filename="gridsearch_5ghz_coll_200hz"
)


--- Running seed 42 ---
Running 160 configurations...

Processing Model: LSTM_HumanFi with params {'input_dim': 52, 'hidden_dim': 64, 'num_layers': 1, 'bidirectional': False, 'dropout': 0.2}
Started loading data...
Train size: 345, Validation size: 62, Test size: 62
Train labels distribution: Counter({8: 21, 0: 20, 1: 20, 2: 20, 13: 20, 14: 20, 15: 20, 16: 20, 3: 19, 4: 19, 6: 19, 10: 19, 11: 19, 12: 19, 5: 18, 7: 18, 9: 17, 17: 17})
Validation labels distribution: Counter({0: 4, 3: 4, 4: 4, 6: 4, 8: 4, 10: 4, 11: 4, 12: 4, 1: 3, 2: 3, 5: 3, 7: 3, 9: 3, 13: 3, 14: 3, 15: 3, 16: 3, 17: 3})
Test labels distribution: Counter({0: 4, 3: 4, 4: 4, 6: 4, 8: 4, 10: 4, 11: 4, 12: 4, 1: 3, 2: 3, 5: 3, 7: 3, 9: 3, 13: 3, 14: 3, 15: 3, 16: 3, 17: 3})
Done loading data (batch size: 32) in 0:00:00.317447.

Testing Configuration 0: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.0, smoothing_prob=0.0

(Device: cuda) Started training model: LSTM_HumanFi
Early stopping at epoch 143 (no improvem

C:\Users\centu\PycharmProjects\MasterThesis_mmWavePI\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Early stopping at epoch 134 (no improvement in 50 epochs)
Training time:  0:01:48.678766


C:\Users\centu\PycharmProjects\MasterThesis_mmWavePI\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)



Testing Configuration 65: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.0, smoothing_prob=0.5

(Device: cuda) Started training model: TemporalConvNet
Early stopping at epoch 169 (no improvement in 50 epochs)
Training time:  0:02:15.596532


...


Testing Configuration 109: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.0, smoothing_prob=0.5

(Device: cuda) Started training model: TemporalConvNet
Early stopping at epoch 161 (no improvement in 50 epochs)
Training time:  0:05:59.498289


C:\Users\centu\PycharmProjects\MasterThesis_mmWavePI\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)



Testing Configuration 110: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.4, smoothing_prob=0.0

(Device: cuda) Started training model: TemporalConvNet
Early stopping at epoch 313 (no improvement in 50 epochs)
Training time:  0:11:38.084630


C:\Users\centu\PycharmProjects\MasterThesis_mmWavePI\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)



Testing Configuration 111: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.4, smoothing_prob=0.5

(Device: cuda) Started training model: TemporalConvNet
Early stopping at epoch 251 (no improvement in 50 epochs)
Training time:  0:09:20.670120

Processing Model: CNN_BiLSTM_Attention with params {'input_dim': 52, 'cnn_filters': 64, 'lstm_units': 64, 'num_layers': 1}
Started loading data...
Train size: 345, Validation size: 62, Test size: 62
Train labels distribution: Counter({8: 21, 0: 20, 1: 20, 2: 20, 13: 20, 14: 20, 15: 20, 16: 20, 3: 19, 4: 19, 6: 19, 10: 19, 11: 19, 12: 19, 5: 18, 7: 18, 9: 17, 17: 17})
Validation labels distribution: Counter({0: 4, 3: 4, 4: 4, 6: 4, 8: 4, 10: 4, 11: 4, 12: 4, 1: 3, 2: 3, 5: 3, 7: 3, 9: 3, 13: 3, 14: 3, 15: 3, 16: 3, 17: 3})
Test labels distribution: Counter({0: 4, 3: 4, 4: 4, 6: 4, 8: 4, 10: 4, 11: 4, 12: 4, 1: 3, 2: 3, 5: 3, 7: 3, 9: 3, 13: 3, 14: 3, 15: 3, 16: 3, 17: 3})
Done loading data (batch size: 32) in 0:00:00.151506.

Testing Confi